<class 'langgraph.prebuilt.tool_executor.ToolExecutor'>


In [ ]:

import os
from git import Repo
from langchain.agents import initialize_agent, load_tools
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI


# Function to clone the repo and list all files
def clone_repo_and_list_files(repo_url, temp_dir):
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    Repo.clone_from(repo_url, temp_dir)

    files_list = []
    for root, _, files in os.walk(temp_dir):
        for file in files:
            files_list.append(os.path.join(root, file))
    return files_list


# Function to download a specific file
def download_file(file_path, download_dir):
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    download_path = os.path.join(download_dir, os.path.basename(file_path))
    os.rename(file_path, download_path)
    print(f"File downloaded successfully to: {download_path}")


# Langchain Prompt to decide which file to download
prompt = PromptTemplate(
    input_variables=["files_list"],
    template="Here is the list of files:\n{files_list}\nWhich file should I download? Return the file path.",
)

# Initialize Langchain LLM
llm = OpenAI(model="text-davinci-003", api_key="your_openai_api_key")

# Define a simple sequential chain
chain = SimpleSequentialChain(
    llm=llm,
    prompt_template=prompt,
)


# Main function
def main(repo_url, temp_dir, download_dir):
    # Step 1: Clone repo and list files
    files_list = clone_repo_and_list_files(repo_url, temp_dir)

    # Step 2: Use Langchain to decide which file to download
    files_list_str = "\n".join(files_list)
    file_to_download = chain.run({"files_list": files_list_str})

    # Step 3: Download the decided file
    file_to_download = file_to_download.strip()  # Clean the output
    if file_to_download in files_list:
        download_file(file_to_download, download_dir)
    else:
        print("Invalid file path returned by Langchain.")


# Example usage
repo_url = "https://github.com/minki-j/GitMeetUp.git"
temp_dir = "./temp_repo"
download_dir = "./downloads"

main(repo_url, temp_dir, download_dir)